In [90]:
import os
import glob
from typing import List
import datetime
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.vectorstores import Chroma, FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()

True

In [75]:
!(app\chatbot\chatbot.png)

O sistema n�o pode encontrar o caminho especificado.


In [76]:
def create_chat(t=0.0):
    """
    Configura os objetos do AzureChatOpenAI para serem utilizados.
 
    Args:
        t (float): O valor da temperatura para determinar o comportamento da resposta, por padrão é 0.0.
   
    Returns:
        AzureChatOpenAI: O objeto de AzureChatOpenAi configurado.
    """
    llm_chat = AzureChatOpenAI(openai_api_base=os.getenv("OPENAI_API_BASE"),
                    openai_api_version=os.getenv("OPENAI_API_VERSION"),
                    openai_api_key=os.getenv("OPENAI_API_KEY"),
                    openai_api_type=os.getenv("OPENAI_API_TYPE"),
                    deployment_name=os.getenv("DEPLOYMENT_NAME"),
                    temperature=t
    )
    return llm_chat

In [77]:
# load pdfs
loaders = [
    PyPDFLoader(r'C:\AI\Cursos\LangChain_Chat_Data\data\Docker_para_desenvolvedores.pdf'),
           PyPDFLoader(r'C:\AI\Cursos\LangChain_Chat_Data\data\Containers_com_Docker.pdf')
        ]   
           
docs = []

for loader in loaders:
    docs.extend(loader.load())

In [78]:
# split

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
)

splits = text_splitter.split_documents(docs)
# Total de chunks
len(splits)

336

In [79]:
def create_embeddings():
    embeddings = OpenAIEmbeddings(
        deployment = os.getenv("EMBEDDING_DEPLOYMENT_NAME"),
    )
    return embeddings

embedding = create_embeddings()

In [80]:
# Armazenamento no VectorDB
vectordb = Chroma.from_documents(
    
    documents=splits,
    embedding=embedding,
    #persist_directory=persist_directory
)

print(vectordb._collection.count())

1008


In [81]:
query = "O que é um docker compose e como configura-lo?"

# Busca por similaridades
docs = vectordb.similarity_search(query,k=3)
len(docs)

3

In [82]:
llm = create_chat()
llm.predict("Hello world")

'Hello there! How can I assist you today?'

In [85]:
# Construir o prompt para auxiliar o modelo na forma e nas informações auxiliares que ajudarão a responder as perguntas.
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
context: {context}
question: {question}
Helpful Answer:"""
prompt = PromptTemplate(template=template,input_variables=['context','question'])

In [86]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever = vectordb.as_retriever(),
    return_source_documents=True,
    # Passando a estrutura que irá auxiliar
    chain_type_kwargs={'prompt':prompt}
)


In [87]:
question = "O que é um docker compose e como configura-lo?"

In [88]:
result = qa_chain({'query':question})
result["result"]

'Docker Compose é uma ferramenta para definição e execução de múltiplos containers Docker, permitindo configurar todos os parâmetros necessários para executar cada container a partir de um arquivo de definição. Esse arquivo é escrito seguindo o formato YAML e pode especificar serviços, volumes e redes a serem criados. A indentação é um fator importante na definição do arquivo e cada linha pode ser definida com uma chave valor ou uma lista. Thanks for asking!'

## Memory

In [91]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
    )

## ConversationalRetrievalChain

In [92]:
qa = ConversationalRetrievalChain.from_llm(
    retriever=vectordb.as_retriever(),
    llm=llm,
    memory=memory
)

In [93]:
question = "O que é uma imagem?"
result = qa({"question": question})

In [95]:
result['answer']

'No contexto do Docker, uma imagem é uma abstração da infraestrutura em estado somente leitura, a partir da qual será instanciado um container. É como se fosse uma classe em orientação a objetos, enquanto o container seria um objeto. Toda vez que um container é iniciado, ele é iniciado a partir de uma imagem. As imagens podem ser oficiais ou não oficiais e podem ser personalizadas para atender às necessidades específicas de um projeto.'

In [96]:
question = "Como posso criar uma?"
result = qa({"question": question})

In [97]:
result['answer']

'Para criar uma imagem no Docker, você precisa criar um arquivo chamado Dockerfile que descreve as etapas necessárias para construir a imagem. O Dockerfile contém uma série de instruções que o Docker segue para criar a imagem. Depois de criar o Dockerfile, você pode usar o comando "docker build" para criar a imagem. Certifique-se de estar no diretório onde o Dockerfile está localizado e execute o comando "docker build -t nome_da_imagem ." (o ponto no final indica que o Docker deve usar o diretório atual como contexto para a construção da imagem).'